### Colab para realizar un data wrangling sobre el [padron de escuelas argentinas](https://datos.gob.ar/ar/dataset/educacion-padron-oficial-establecimientos-educativos) con el fin alimentar colecciones divididas segun las jurisdicciones provinciales y asi permitir que una app de firebase cruzara datos desde la [API del Servicio de Normalización de Datos Geográficos de Argentina](https://datosgobar.github.io/georef-ar-api/) con las diversas escuelas del territorio nacional.

In [ ]:
!pip install firebase-admin

In [ ]:

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io


___
#CARGA DB Y SUBDIVICION 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving escuelas.csv to escuelas.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['escuelas.csv']), header=5)

secundarias=df[ (df['Secundaria']==1) |(df['Secundaria.1']==1) | (df['Secundaria.2']==1)| (df['Secundaria.3']==1)| (df['Secundaria.4']==1)|(df["Secundaria Técnica (INET)"]==1)|(df["EGB3"]==1)|(df["EGB3"]==1)]

df_short=secundarias.drop(secundarias.columns[15:], axis=1, inplace=False)

df_short.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17677 entries, 0 to 63649
Data columns (total 15 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Jurisdicción                                       17677 non-null  object
 1   Sector                                             17677 non-null  object
 2   Ámbito                                             17677 non-null  object
 3   Departamento                                       17677 non-null  object
 4   Código de departamento                             17677 non-null  int64 
 5   Localidad                                          17677 non-null  object
 6   Código localidad                                   17677 non-null  int64 
 7   Clave única de establecimiento (cue)               17677 non-null  int64 
 8   Clave única de establecimiento - anexo (cueanexo)  17677 non-null  int64 
 9   Nombre           

In [ ]:
listaRenombrar =['jurisdiccion', 'sector', 'ambito','departamento','codigoDepartamento','localidad','codigoLocalidad','cue','anexoCue','nombre','domicilio','codigoPostal','codigoTelefonico','telefono','email' ]

df_short = df_short.set_axis(listaRenombrar, axis='columns')

jurisdicciones =list(dict.fromkeys(df['Jurisdicción'].values.tolist()))
print(len(jurisdicciones))
print( jurisdicciones)

dbs={} # buscar un mejor nombre
for jurisdiccion in jurisdicciones:
    dbs[jurisdiccion]=df_short[ df_short['jurisdiccion']==jurisdiccion].to_dict('records') 

24
['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Córdoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán', 'Tierra del Fuego']


---
#CARGA FIREBASE

In [ ]:
keyFire ={
    # pegar key de la app firebase o subir json
}


In [ ]:

cred = credentials.Certificate(keyFire)
firebase_admin.initialize_app(credential=cred) 
db = firestore.client()

In [ ]:
#lista de escuelas a cargar 
jurisdicciones = ['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Córdoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán', 'Tierra del Fuego']


#en caso de necsitar la totalidad de las escuelas argentinas

# se cuenta por archivo grabado por ende solo puedo subir de a 20k diarios sin sobrepasar el limite gratuito
# 63650 registros en la db (aprox restando bs as 14.690 subidas al dia)
"""
jurisdicciones_dia1 = ['Ciudad de Buenos Aires', 'Catamarca', 'Córdoba', 'Corrientes', 'Chaco']
jurisdicciones_dia2 = ['Buenos Aires']
jurisdicciones_dia3 = ['Chubut', 'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquén']
jurisdicciones_dia4 = ['Río Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán', 'Tierra del Fuego']
"""

# adaptacion del diccionario con las seccionnes del df a los nombres compatibles con firebase en formato camelCase
nombreCollections ={'Ciudad de Buenos Aires':'caba', 'Buenos Aires':"buenosAires", 'Catamarca':'catamarca', 'Córdoba':'cordoba', 'Corrientes':'corrientes', 'Chaco':'chaco', 'Chubut':'chubut', 'Entre Ríos':'entreRios', 'Formosa':'formosa', 'Jujuy':'jujuy', 'La Pampa':'laPampa', 'La Rioja':'laRioja', 'Mendoza':'mendoza', 'Misiones':'misiones', 'Neuquén':'neuquen', 'Río Negro':'rioNegro', 'Salta':'salta', 'San Juan':'sanJuan', 'San Luis':'sanLuis', 'Santa Cruz':'santaCruz', 'Santa Fe':'santaFe', 'Santiago del Estero':'santiagoDelEstero', 'Tucumán':'tucuman', 'Tierra del Fuego':'tierraDelFuego'}
Collections =['caba',"buenosAires",'catamarca','cordoba', 'corrientes', 'chaco', 'chubut', 'entreRios', 'formosa', 'jujuy', 'laPampa', 'laRioja', 'mendoza', 'misiones','neuquen', 'rioNegro', 'salta', 'sanJuan','sanLuis','santaCruz','santaFe','santiagoDelEstero', 'tucuman','tierraDelFuego']



In [ ]:

#reglas para las colecciones de escuelas (al combinar reglas de varias partes de proyectos preferi no subirlas automaticamente)
for coleccion in Collections:
  print("match /"+coleccion+"/{document} {\nallow read: if ture;\nallow write: if false;\nallow delete : if false;\n}")

In [ ]:
batch = db.batch()
contador=0
#en caso de ser por dias for jurisdiccion in jurisdiccione_dia n°:
for jurisdiccion in jurisdicciones:
  for escuela in dbs[jurisdiccion]:
    contador+=1
    ref_db = db.collection(nombreCollections[jurisdiccion]).document() 
    # seteo documento en un bacth
    batch.set(ref_db, escuela)
    if(contador >=499):
      contador=0
      batch.commit() 
      batch = db.batch()

if(contador!=0): #commit para el ultimo bloque del for
  batch.commit()

In [ ]:
# como se subirian de ser la totalidad de escuelas

# Ciudad de Buenos Aires 2804 -d1
# Catamarca 1090 -d1
# Córdoba 5440 -d1
# Corrientes 2049 -d1
# Chaco 3157 -d1

# Buenos Aires 19579 -d2

# Chubut 827 -d3
# Entre Ríos 2870 -d3
# Formosa 1703 -d3
# Jujuy 1013 -d3
# La Pampa 742 -d3
# La Rioja 893 -d3
# Mendoza 2912 -d3
# Misiones 2789 -d3
# Neuquén 917 -d3

# Río Negro 1286 -d4
# Salta 1514 -d4
# San Juan 1356 -d4
# San Luis 689 -d4
# Santa Cruz 386 -d4 
# Santa Fe 5003 -d4
# Santiago del Estero 2427 -d4 
# Tucumán 1991 -d4 
# Tierra del Fuego 213 -d4